In [11]:
import imp
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [12]:
# INPUT 
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Qty 4 Months/Input.xlsx",sheet_name="Input").rename(columns={"FOB Price (USD/Pcs)":"Unit Price"})
# Exchange Rate
data_exchange_rate=pd.read_excel("../VACP/Python NCC Min/Vietcombank Exchange Currency.xlsx",usecols="B,E").rename(columns={"Giá bán VND":"Tỷ giá VND"})

# # Tinh Unit Price VND
data_input=data_input.merge(data_exchange_rate,on="Mã nguyên tệ")
Exchange_Cost=0.05

data_input["Unit Price VND"] = np.round((1+Exchange_Cost)*data_input["Unit Price"]*data_input["Tỷ giá VND"],0)
data_input=data_input.drop(columns=["No.","Tỷ giá VND"])
data_input.head()


,Brand,Order Conf. Date,Calculated Date,Order Ref.,PO Number,ID,Substitution,Description,Order Qty,Unit Price,Mã nguyên tệ,Total Order Amount,Agt 22 Ready Stock,Số lượng cần nhập,Số lượng chênh lệch,Note,Unit Price VND
0,Denso,2022-01-01,2022-08-22,NaN,NaN,065000-2940ND AC,NaN,Motor Assy,20,31.85,USD,637.00,0.0,20.0,0.0,Order,803623.0
1,Denso,2022-01-01,2022-08-22,NaN,NaN,168000-1070ND AC,NaN,Motor Fan,20,18.35,USD,367.00,0.0,20.0,0.0,Order,462998.0
2,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,15,24.99,USD,374.85,0.0,20.0,-5.0,Order,630535.0
3,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,20,24.99,USD,499.80,0.0,20.0,0.0,Order,630535.0
4,Toyota,2022-04-26,2022-08-22,NaN,NaN,13011-0L030,NaN,"RING SET, PISTON",4,45.83,USD,183.32,0.0,4.0,0.0,Order,1156360.0


In [13]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4).rename(columns={"Mã số":"ID"}).dropna(
                subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M").rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_19164\1085106072.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={


In [14]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =np.round(
                                         (1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            ))
                                    ,0)

In [15]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [16]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID').merge(data_giaban,on="ID")
data_report=data_input.merge(data_nhap_ban[["ID","Giá vốn","Giá bán","Giá bán giảm","Tổng nhập","Tổng xuất","Tổng tồn"]], on="ID", how="left")


In [17]:
data_report.loc[1:30,["ID","Order Conf. Date","Calculated Date","Unit Price VND","Giá vốn","Giá bán giảm","Số lượng cần nhập","Agt 22 Ready Stock","Tổng xuất","Tổng tồn"]].sort_values(
                                                by=["Tổng tồn","ID"],ascending=False).reset_index(drop=True)

,ID,Order Conf. Date,Calculated Date,Unit Price VND,Giá vốn,Giá bán giảm,Số lượng cần nhập,Agt 22 Ready Stock,Tổng xuất,Tổng tồn
0,08889-80280,2022-06-07,2022-08-22,147352.0,151673.0,200000.0,10000.0,1200.0,7182.0,3620.0
1,90919-T1004,2022-06-07,2022-08-22,20185.0,20701.0,25000.0,10000.0,560.0,38961.0,2827.0
2,31210-0D240,2022-06-07,2022-08-22,447607.0,507965.0,650000.0,800.0,5.0,1457.0,345.0
3,31210-0K310,2022-06-07,2022-08-22,996897.0,1063137.0,1190000.0,450.0,0.0,779.0,321.0
4,31210-0K300,2022-06-07,2022-08-22,878561.0,936034.0,1050000.0,300.0,0.0,464.0,198.0
5,13408-0C030,2022-06-07,2022-08-22,862665.0,879840.0,1200000.0,300.0,54.0,388.0,189.0
6,31210-0K040,2022-06-07,2022-08-22,1325663.0,1346232.0,1550000.0,150.0,24.0,365.0,169.0
7,90916-T2039,2022-06-07,2022-08-22,230868.0,274560.0,430000.0,1000.0,162.0,1315.0,121.0
8,31210-0A010,2022-06-07,2022-08-22,713799.0,726098.0,950000.0,250.0,0.0,326.0,114.0
9,31210-0K101,2022-06-07,2022-08-22,951732.0,979218.0,1090000.0,1000.0,0.0,1914.0,109.0


In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/Analysing Qty 4 Months/Report.xlsx') as writer:
    data_report.to_excel(writer,sheet_name="Tổng hợp")
